In [1]:
import sys
sys.path.insert(0, 'C:/Users/14432/OneDrive/Research/Projects/A549_144hr/src/4_wk_trajectory/')
sys.path.insert(1, 'C:/Users/14432/OneDrive/Research/Projects/A549_144hr/src/memes/')

import warnings
warnings.filterwarnings("ignore")

import numpy as np
from skimage import measure
from skimage.segmentation import find_boundaries
from skimage.morphology import opening,closing
from skimage.io import imread
from matplotlib import pyplot as plt
import os
from os import listdir
import pandas as pd
from scipy.stats import kde
import seaborn as sns
import copy
from math import exp,log
import pickle
import scipy.ndimage as ndimage
import scipy.interpolate.fitpack as fitpack
from sklearn import manifold,decomposition,random_projection,cluster,metrics,preprocessing,mixture,model_selection
from sklearn.neighbors import kneighbors_graph,BallTree
from hmmlearn import hmm
# from pymc import MCMC,flib,Model,MAP
# from ripser import Rips,ripser,plot_dgms
from persim import PersImage
import scipy.io as sio
# import kmapper as km
# from kmapper import jupyter
from mpl_toolkits.mplot3d import Axes3D
from cell_class import single_cell,fluor_single_cell
import contour_class
import utility_tools
import image_warp
from contour_tool import df_find_contour_points,find_contour_points,generate_contours,align_contour_to,align_contours
from scipy.signal import medfilt,wiener
from traj_class import single_cell_traj,fluor_single_cell_traj
# import bnpy
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler,QuantileTransformer
from itertools import product
from sklearn.cluster import DBSCAN,KMeans
from sklearn.metrics import silhouette_score,davies_bouldin_score
import glob
# from pykalman import KalmanFilter
# from neupy import algorithms, utils
from tslearn.utils import to_time_series,to_time_series_dataset
from tslearn.clustering import TimeSeriesKMeans,KShape,GlobalAlignmentKernelKMeans,silhouette_score
from tslearn.metrics import dtw_path,dtw_subsequence_path,cdist_gak,cdist_dtw
from tslearn.barycenters import dtw_barycenter_averaging, softdtw_barycenter
from sklearn.manifold import MDS
# import sparse
from statsmodels.tsa.stattools import grangercausalitytests
from scipy import signal,stats
from statsmodels import robust
from matplotlib.patches import Circle, Wedge, Rectangle
from traj_scale import sp_traj_scaling,ssp_traj_scaling

In [2]:
main_path='C:/Users/14432/OneDrive/Research/Projects/A549_144hr/data/'
output_path=main_path+'out/11-2-21_mt_20E-3_lt_35E-2/'
sct_path=output_path+'sct/'
result_path=output_path+'result/'
#sct_list=sorted(listdir(sct_path))
sct_list=sorted(glob.glob(sct_path+'*long_traj*'))
print(len(sct_list))

505


In [3]:
with open (main_path+'scale_morph_pca', 'rb') as fp:
    morph_pca=pickle.load(fp)
with open (main_path+'vimentin_haralick_scaler', 'rb') as fp:
    vim_scaler=pickle.load(fp)
with open (main_path+'vimentin_haralick_scale_pca', 'rb') as fp:
    vim_pca=pickle.load(fp)

# with open (main_path+'EM_scaler', 'rb') as fp:
#     EM_scaler=pickle.load(fp)

In [4]:
key_list=['Cell_AreaShape_Area', 'Cell_AreaShape_Center_X', 'Cell_AreaShape_Center_Y', 'Cell_AreaShape_Center_Z', \
          'Cell_AreaShape_Compactness', 'Cell_AreaShape_Eccentricity', 'Cell_AreaShape_EulerNumber', \
          'Cell_AreaShape_Extent', 'Cell_AreaShape_FormFactor', 'Cell_AreaShape_MajorAxisLength',\
          'Cell_AreaShape_MaxFeretDiameter', 'Cell_AreaShape_MaximumRadius', 'Cell_AreaShape_MeanRadius', \
          'Cell_AreaShape_MedianRadius', 'Cell_AreaShape_MinFeretDiameter', 'Cell_AreaShape_MinorAxisLength', \
          'Cell_AreaShape_Orientation', 'Cell_AreaShape_Perimeter', 'Cell_AreaShape_Solidity']
key_mask=np.ones((len(key_list),),dtype=bool)
key_mask[key_list.index('Cell_AreaShape_Center_Y')]=False
key_mask[key_list.index('Cell_AreaShape_Center_X')]=False
key_mask[key_list.index('Cell_AreaShape_Center_Z')]=False
key_mask[key_list.index('Cell_AreaShape_EulerNumber')]=False
key_mask[key_list.index('Cell_AreaShape_Orientation')]=False
print(key_list.index('Cell_AreaShape_Center_X'),key_list.index('Cell_AreaShape_Center_Y'),key_list.index('Cell_AreaShape_Center_Z'),key_list.index('Cell_AreaShape_EulerNumber'))

1 2 3 6


In [ ]:
wd_size=12
scale_features=[]
scale_contours=[]
scale_vim=[]
fluor_name='vimentin_scale_haralick'
for i in range(len(sct_list)):
    with open (sct_list[i], 'rb') as fp:
        sct = pickle.load(fp)
    print(sct_list[i])
    mask=sct.traj_vimentin_feature_values[0]!=0
    inds=np.where(sct.traj_vimentin_feature_values[0]!=0)
    traj_t=sct.traj_seri[mask][:,0]
    traj_cord=sct.traj_cord[mask]
    traj_vim=sct.traj_vimentin_haralick_pca_cord[mask]
    
#     dot_color=np.arange(traj_t.shape[0])
#     cm=plt.cm.get_cmap('jet')
#     sc=plt.scatter(traj_cord[:,0],traj_vim[:,0],c=dot_color,cmap=cm)
#     plt.colorbar(sc)
#     plt.show()

    
    if np.mean(traj_cord[:36,0])<np.mean(traj_cord[-36:,0]) or np.mean(traj_vim[:36,0])<np.mean(traj_vim[-36:,0]):
   
        scale_contour,scale_contour_with_vim,scale_haralick,scale_t=sp_traj_scaling(sct,t_cutoff=6,t_range=48)
    #     scale_contour,scale_contour_with_vim,scale_haralick,morph_scale_t,vim_scale_t=ssp_traj_scaling(sct)
        print(len(scale_contour_with_vim),len(scale_haralick))

        if len(scale_contour)>0:
            scale_morph_cord=morph_pca.transform(scale_contour)
            scale_vim_cord0=vim_pca.transform(vim_scaler.transform(scale_haralick))
            scale_vim_cord=np.zeros((scale_morph_cord.shape[0],scale_vim_cord0.shape[1]))
            print(scale_vim_cord[inds].shape,scale_vim_cord0.shape)
            scale_vim_cord[inds,:]=scale_vim_cord0


            sct.set_traj_scale_cord(scale_morph_cord)
            sct.set_traj_fluor_pca_cord(fluor_name,scale_vim_cord)
            
#             dot_color=np.arange(traj_t.shape[0])
#             cm=plt.cm.get_cmap('jet')

#             plt.scatter(scale_morph_cord[mask][:,0],scale_vim_cord[mask][:,0],c=dot_color,cmap=cm)
#             plt.show()

    #         plt.scatter(scale_morph_cord[mask][:,0],sct.traj_vimentin_haralick_pca_cord[mask][:,0],c=dot_color,cmap=cm)
    #         plt.show()



    #         sc=plt.scatter(scale_vim_cord[mask][:,0],scale_vim_cord[mask][:,1],c=dot_color,cmap=cm)
    #         plt.colorbar(sc)
    #         plt.show()

            with open (sct_list[i], 'wb') as fp:
                pickle.dump(sct,fp)
            print('scaled cord saved on cell ' + str(i))

        else:
            sct.set_traj_scale_cord(None)
            sct.set_traj_fluor_pca_cord(fluor_name,None)


            with open (sct_list[i], 'wb') as fp:
                pickle.dump(sct,fp)


In [6]:

# def cal_sma(X,window_size=12):
#     sma=[np.mean(X[i:i+window_size,:],axis=0) for i in range(X.shape[0]-window_size)]
#     sma=np.array(sma)
#     return sma

# def slide_std(X,window_size=12):
#     sstd=[np.std(X[i:i+window_size,:],axis=0) for i in range(X.shape[0]-window_size)]
#     sstd=np.array(sstd)
#     return sstd

# def slide_mad(X,window_size=12):
#     smad=[robust.scale.mad(X[i:i+window_size,:],c=1,axis=0) for i in range(X.shape[0]-window_size)]
#     smad=np.array(smad)
#     return smad

##this function is scale all the traj contour
# def traj_scaling(sct,wd_size=12,t_range=72,sort_range=6):
    


#     mask=sct.traj_vimentin_feature_values[0]!=0
#     traj_t=sct.traj_seri[mask][:,0]
#     traj_cord=sct.traj_cord[mask]
#     traj_vim=sct.traj_vimentin_haralick_pca_cord[mask]
    
#     morph_sma=cal_sma(traj_cord,window_size=wd_size)

#     morph_sstd=slide_std(traj_cord,window_size=wd_size)
    
#     morph_smad=slide_mad(traj_vim,window_size=wd_size)
    
#     sort_morph_std_ind=np.argsort(np.sum(morph_sstd[:t_range,:],axis=1))[:sort_range]

# #     print(np.amin(morph_sma[sort_morph_std_ind,0]))
        
#     morph_scale_t=sort_morph_std_ind[np.argmin(morph_sma[sort_morph_std_ind,0])]
    
#     morph_scale_area=cal_sma(sct.traj_feature[mask][:,key_mask])[morph_scale_t,0]
#     morph_scale_ar=cal_sma(sct.traj_feature[mask][:,key_mask])[morph_scale_t,8]#mean redius
#     morph_scale_mr=cal_sma(sct.traj_feature[mask][:,key_mask])[morph_scale_t,9]#median redius
    
#     scale_contour=sct.traj_contour/np.sqrt(morph_scale_area)
    
    
#     vim_mi_sma=cal_sma(sct.traj_vimentin_feature_values[0][mask][:,None],window_size=wd_size)
#     vim_sma=cal_sma(sct.traj_vimentin_feature_values[2][mask],window_size=wd_size)
#     vim_sstd=slide_std(sct.traj_vimentin_feature_values[0][mask][:,None],window_size=wd_size)
#     sort_vim_std_ind=np.argsort(np.sum(vim_sstd[:t_range,:],axis=1))[:sort_range]
#     vim_scale_t=sort_vim_std_ind[np.argmin(vim_mi_sma[sort_vim_std_ind,0])]
    
#     vim_scale=vim_sma[vim_scale_t,:]
#     print(morph_scale_t,vim_scale_t)
#     scale_haralick=sct.traj_vimentin_feature_values[2][mask]-vim_scale
    
    
    
    
#     start_morph=morph_pca.transform((cal_sma(sct.traj_contour[mask])[morph_scale_t]/np.sqrt(morph_scale_area))[None,:])
#     start_vim=vim_pca.transform(vim_scaler.transform(cal_sma(scale_haralick)))

    
#     return scale_contour,scale_haralick,start_morph,start_vim

In [7]:
# def consecutive_arrs(data, stepsize=3):
#     return np.split(data, np.where(np.diff(data)>stepsize)[0]+1)
 
# def cluster_points(X, mu):
#     cluster_labels=np.zeros((X.shape[0]))
#     for x in X:
#         bestmukey = min([(i[0], np.linalg.norm(x-mu[i[0]])) for i in enumerate(mu)], key=lambda t:t[1])[0]
#         cluster_labels[np.where(X==x)[0]]=bestmukey

#     return cluster_labels
# def sliding_window_fluct(X,start_t=6):
    
#     Y1=[np.log(np.std(X[:i])) for i in range(start_t,X.shape[0])]
#     Y1=np.array(Y1)
#     plt.plot(Y1)
#     plt.show()
#     Y2=[np.mean(abs(signal.detrend(X[:i]))) for i in range(start_t,X.shape[0])]
#     Y2=np.array(Y2)
#     plt.plot(Y2)
#     plt.title('mean_detrend_fluct')
#     plt.show()
    
#     Z=X[start_t:]/Y1
#     return Z
# def cal_msd(X,start_t=12):
#     ini=np.mean(X[:start_t])
#     msd=[np.sqrt(np.mean((X[:i]-ini)**2)) for i in range(start_t,X.shape[0])]
#     msd=np.array(msd)
#     return msd

# def cal_std(X,start_t=12):
#     sstd=[np.std(X[:i]) for i in range(start_t,X.shape[0])]
#     sstd=np.array(sstd)
#     return sstd

# def cal_rstd(X,start_t=12):
#     ini=np.mean(X[:start_t])
#     rstd=[np.std(X[:i])/np.mean(X[:i]) for i in range(start_t,X.shape[0])]
#     rstd=np.array(rstd)
#     return rstd


# def cal_ste(X,start_t=12):
#     ini=np.mean(X[:start_t])
#     rste=[np.std(X[:i])/np.sqrt(i) for i in range(start_t,X.shape[0])]
#     rste=np.array(rste)
#     return rste
    
    
# def slide_msd(X,window_size=12):
#     msd=[np.sqrt(np.mean((signal.detrend(X[i:i+window_size]-X[i])**2))) for i in range(X.shape[0]-window_size)]
#     msd=np.array(msd)
#     return msd


    

In [8]:
# all_morph_cords=[[] for i in range(577)]
# all_haralick=[[] for i in range(577)]


            
# mean_count=np.zeros((577,))
# mean_vim=np.zeros((577))
# mean_traj_contour=np.zeros((577,300))

# for i in range(len(sct_list)):
#     with open (sct_list[i], 'rb') as fp:
#         sct = pickle.load(fp)
#     for j in range(len(sct.traj_contour)):
#         if np.any(sct.traj_cord[j,:]) and np.any(sct.traj_vimentin_norm_haralick_pca_cord[j,:]):
#             img_num=(sct.traj_seri[j,0]).astype(np.int)
#             mean_traj_contour[img_num-1,:]+=sct.traj_contour[j,:]
#             mean_vim[img_num-1]+=sct.traj_vimentin_feature_values[0][j]
#             mean_count[img_num-1]+=1
#             all_morph_cords[j].append(sct.traj_cord[j,:])
#             all_haralick[j].append(sct.traj_vimentin_norm_haralick_pca_cord[j,:])
            
            

# mean_vim=mean_vim/(mean_count+1e-8)          
# mean_traj_contour=mean_traj_contour/(mean_count[:,None]+1e-8)            
# mean_mask=mean_vim!=0
# print(mean_mask.shape)
# for k in range(577):
#     plt.plot(mean_traj_contour[k,::2],mean_traj_contour[k,1::2],'.')
# plt.show() 


            
# std_morph=np.zeros((577,8))
# std_haralick=np.zeros((577,6))
# mean_morph=np.zeros((577,8))
# mean_haralick=np.zeros((577,6))

# median_morph=np.zeros((577,8))
# median_haralick=np.zeros((577,6))
# mad_morph=np.zeros((577,8))
# mad_haralick=np.zeros((577,6))

# for k in range(len(all_morph_cords)):
#     if len(all_morph_cords[k])>0:
#         mean_morph[k,:]=np.mean(np.vstack(all_morph_cords[k]),axis=0)
#         std_morph[k,:]=np.std(np.vstack(all_morph_cords[k]),axis=0)
#         mean_haralick[k,:]=np.mean(np.vstack(all_haralick[k]),axis=0)
#         std_haralick[k,:]=np.std(np.vstack(all_haralick[k]),axis=0)
        
#         median_morph[k,:]=np.median(np.vstack(all_morph_cords[k]),axis=0)
#         median_haralick[k,:]=np.median(np.vstack(all_haralick[k]),axis=0)
#         mad_morph[k,:]=robust.scale.mad(np.vstack(all_morph_cords[k]),c=1,axis=0)
#         mad_haralick[k,:]=robust.scale.mad(np.vstack(all_haralick[k]),c=1,axis=0)


   
# dot_color=np.arange(mean_vim[mean_mask].shape[0])
# cm=plt.cm.get_cmap('jet')

# sc=plt.scatter(mean_morph[mean_mask][:,0],mean_haralick[mean_mask][:,0],c=dot_color,cmap=cm)
# plt.xlabel('PC1',fontsize=16)
# plt.ylabel('PC2',fontsize=16)
# #     plt.axis([-450,800,-400,400])
# plt.colorbar(sc)
# plt.show()

# sc=plt.scatter(mean_morph[mean_mask][:,0],mean_morph[mean_mask][:,1],c=dot_color,cmap=cm)
# plt.xlabel('PC1',fontsize=16)
# plt.ylabel('PC2',fontsize=16)
# #     plt.axis([-450,800,-400,400])
# plt.colorbar(sc)
# plt.show()

# sc=plt.scatter(mean_haralick[mean_mask][:,0],mean_haralick[mean_mask][:,1],c=dot_color,cmap=cm)
# plt.xlabel('PC1',fontsize=16)
# plt.ylabel('PC2',fontsize=16)
# #     plt.axis([-450,800,-400,400])
# plt.colorbar(sc)
# plt.show()

# plt.plot(std_morph[mean_mask][:12,0])
# plt.show()

# plt.plot(mad_morph[mean_mask][:12,0])
# plt.show()

# morph_start_range=[np.amin(median_morph[mean_mask][0,0]-mad_morph[mean_mask][0,0]),np.amax(median_morph[mean_mask][0,0]+mad_morph[mean_mask][0,0])]
# vim_start_range=[np.amin(median_haralick[mean_mask][0,0]-mad_haralick[mean_mask][0,0]),np.amax(median_haralick[mean_mask][0,0]+mad_haralick[mean_mask][0,0])]
# print(morph_start_range,vim_start_range)

In [9]:
# Y=[]
# Y1=[]
# Y2=[]
# for i in range(len(sct_list)):
#     with open (sct_list[i], 'rb') as fp:
#         sct = pickle.load(fp)
#     mask=sct.traj_vimentin_feature_values[0]!=0

#     traj_t=sct.traj_seri[mask][:,0]
# #     X=sct.traj_cord[mask][:,:2]
# #     X=sct.traj_vimentin_norm_haralick_pca_cord[mask][:,:3]
# #     X=sct.traj_vimentin_norm_haralick_pca_cord[mask][:,:3]
#     X0=np.column_stack((sct.traj_cord[mask][:,:],sct.traj_vimentin_norm_haralick_pca_cord[mask][:,:]))
    

# #     print(sct.traj_cord[mask][:,:].shape)

#     X=np.column_stack((X0[:,0],X0[:,8]))
# #     qt = Qustatsmodels.robust.scale.mad¶antileTransformer(n_quantiles=100, output_distribution='normal')
# #     X=qt.fit_transform(X)
# #     X=RobustScaler(with_scaling=False, quantile_range=(10.0, 90.0)).fit_transform(X)
# #     Y.append(X)
     
# #     dot_color=np.arange(X.shape[0])
# #     cm=plt.cm.get_cmap('jet')
# #     plt.scatter(X[:,0],X[:,1],c=dot_color,cmap=cm)

# #     plt.show()
    
# #     plt.hist(X[:,0],bins=20)
# #     plt.show()
# #     plt.hist(X[:,1],bins=20)
# #     plt.show()
#     for j in range(X.shape[1]):
#         X[:,j]=(X[:,j]-np.median(X[:,j]))
    
# #     X_trend=np.zeros_like(X)
# #     for j in range(X.shape[1]):
# #         X_trend[:,j]= signal.medfilt(X[:,j], 7)
# # #     plt.plot(X[:,0],color='b')
# # #     plt.plot(X_trend[:,0],color='r')
# # #     plt.show()
# #     X_fluct=X-X_trend
# #     print(np.std(X,axis=0))
# #     print(np.std(X_fluct,axis=0))
    
# #     scale_F=np.std(X,axis=0)[:,None]/np.std(X_fluct,axis=0)[:,None]
    
    
#     scale_X=X
#     Y.append(scale_X)
#     plt.scatter(scale_X[:,0],scale_X[:,1],s=0.01)
# plt.show()

# sns.kdeplot(np.vstack(Y)[:,0],np.vstack(Y)[:,1],n_levels=50,shade=True)
# # plt.axis([-300,300,-2.5,2.5])
# plt.show()

#     morph_norm=np.linalg.norm(sct.traj_contour,axis=1)[mask]
# #     vim_norm=np.linalg.norm(sct.traj_vimentin_feature_values[3],axis=1)[mask]
# #     Y=sliding_window_fluct(morph_norm)
# #     plt.hist(morph_norm,bins=30)
# #     plt.show()
    
#     msd=cal_msd(morph_norm)
#     plt.plot(msd)
#     plt.show()
    
#     sstd=cal_std(morph_norm)
#     plt.plot(sstd)
#     plt.show()
    
#     rstd=cal_rstd(morph_norm)
#     plt.plot(rstd)
#     plt.show()
    
#     rste=cal_ste(morph_norm)
#     plt.plot(rste)
#     plt.show()
    

In [10]:
#     plt.hist(vim_norm,bins=30)
#     plt.show()
    
    

    
#     s_msd=slide_msd(morph_norm)
#     plt.plot(s_msd)
#     plt.show()
    
#     X_cumsum=np.cumsum(morph_norm,axis=0)
#     plt.plot(X_cumsum)
#     plt.show()
    
    
    
#     X_trend=np.zeros_like(X)
#     for j in range(X.shape[1]):
#         X_trend[:,j]= signal.medfilt(X[:,j], 7)
# #     plt.plot(X[:,0],color='b')
# #     plt.plot(X_trend[:,0],color='r')
# #     plt.show()
#     X_fluct=X-X_trend
    
#     X_cumsum=np.cumsum(X_fluct,axis=0)
        
    
#     dot_color=np.arange(X_cumsum.shape[0])
#     cm=plt.cm.get_cmap('jet')
#     plt.scatter(np.arange(X_cumsum.shape[0]),X_cumsum[:,0],s=5,c=dot_color,cmap=cm)
#     plt.show()
    
#     plt.scatter(np.arange(X_cumsum.shape[0]),X_cumsum[:,1],s=5,c=dot_color,cmap=cm)
#     plt.show()
    
#     plt.plot(X_fluct[:,0])
#     plt.show()
#     plt.plot(X_fluct[:,1])
#     plt.show()
                              
    
#     Y1.append(np.mean(X,axis=0))
#     Y2.append(np.mean(abs(X_fluct),axis=0))
    
    
    
#     X_new=X.copy()
#     X_new[:,0]=X[:,0]/np.mean(abs(X_fluct[:,0]))

#     X_new[:,1]=X[:,1]/np.mean(abs(X_fluct[:,1]))   
    
#     Y.append(X_new)
#     dot_color=np.arange(X.shape[0])
#     cm=plt.cm.get_cmap('jet')
#     plt.scatter(X_new[:,0],X_new[:,1],c=dot_color,cmap=cm)
# #     plt.axis([-25,25,-4,8])
#     plt.show()


#     lnX=np.log(X+100)
#     plt.plot(lnX[:,0],color='b')
#     plt.show()
#     traj_proba=gmm.predict_proba(X)
#     traj_state=gmm.predict(X)
#     traj_state[traj_proba[:,1]>0.6]=2
#     traj_state[traj_proba[:,1]<0.4]=0
#     traj_state[(traj_proba[:,1]>=0.4)&(traj_proba[:,1]<=0.6)]=1

# #     mds=manifold.MDS(n_components=2, metric=False)
# #     X_nmds=mds.fit_transform(X)
# #     plt.scatter(X_nmds[:,0],X_nmds[:,1],c=dot_color,cmap=cm)
# #     plt.show()

    
#     E_arrs=consecutive_arrs(np.where(traj_state==0)[0])
#     M_arrs=consecutive_arrs(np.where(traj_state==2)[0])
#     for m in range(len(E_arrs)):
#         if E_arrs[m].shape[0]>24:
#             reaction_start=E_arrs[m][0]
#             break
#     n=len(M_arrs)
#     while n>=0:
#         n-=1
#         if M_arrs[n].shape[0]>24:
#             reaction_end=M_arrs[n][-1]
#             break
#     if reaction_start<72 and reaction_end>145:
#         print(reaction_start,reaction_end)
        
#         reaction_traj_fd=X0[reaction_start:reaction_end+1,:]
#         reaction_traj=X[reaction_start:reaction_end+1,:]
#         reaction_t=traj_t[reaction_start:reaction_end+1]
        
#         reaction_grad=np.gradient(reaction_traj,axis=0)/np.expand_dims(np.gradient(reaction_t),axis=1)
#         reaction_grad[0]=0
#         abs_reaction_grad=abs(np.linalg.norm(reaction_grad,axis=1))
        
        
#         dot_color=np.arange(reaction_traj.shape[0])
#         cm=plt.cm.get_cmap('jet')
#         plt.scatter(reaction_traj[:,0],reaction_traj[:,1],c=dot_color,cmap=cm)
#         plt.xlabel('Morphology PC1',fontsize=16)
#         plt.ylabel('Vimentin Haralick PC1',fontsize=16)
#         plt.show()